# AGH Modelling of Physical Systems Lab 4 (2025 March 25th)
Sylwester Arabas (sylwester.arabas@agh.edu.pl)

**1. diffusion PDE**

Let's introduce a few symbols:   
* ${\color{red}t}$: time   
* ${\color{blue}x}$: physical coordinate (e.g., distance or altitude)   
* ${\color{green}\psi}({\color{blue}x})$: scalar field (e.g., pollutant concentration)   

Transport of $\color{green}\psi$ due to diffusion with consant coefficient $\color{orange}\mu$ can be modelled with the diffusion equation:
$$ \partial_{\color{red} t} {\color{green}\psi} - {\color{orange}\mu} \partial^2_{\color{blue}x}{\color{green}\psi}= 0 $$

For the special case of domain of length $L$ and initial condition ${\color{green}\psi}({\color{red} t}=0,{\color{blue}x}) = \sin(\pi {\color{blue}x} / L)$, an analytical solution is readily available:
$$ {\color{green}\psi}({\color{blue}x}, {\color{red}t})=e^{-\mu (\pi/L)^2 {\color{red} t}} \sin(\pi {\color{blue}x} / L)$$

Let's depict the problem using NumPy and matplotlib:

In [ ]:
import numpy as np
from matplotlib import pyplot
from open_atmos_jupyter_utils import show_plot, show_anim

In [ ]:
class PsiAnalytic:
    def __init__(self, *, mu, L):
        self.mu = mu
        self.L = L
    def __call__(self, *, x, t):
        return np.nan  # TODO

In [ ]:
psi_analytic = PsiAnalytic(mu=.5, L=10)
nx = 64
x, dx = np.linspace(0, psi_analytic.L, nx, endpoint=True, retstep=True)
x = x[:-1] + dx/2
t_max = 10

In [ ]:
pyplot.plot(x, psi_analytic(x=x,t=0), label='initial')
pyplot.plot(x, psi_analytic(x=x,t=t_max), label=f'analytical at {t_max=}')
pyplot.legend()
pyplot.grid()
show_plot('analytic.pdf')

**2. overview of applications**

* diffusion of heat
* diffusion of pollutants in atmosphere and hydrosphere
* diffusion of moisture away from evaporating rain drops
* diffusion of in PDE originating from SDE in mathematical finance (market volatility)
* ...

**3. simple numerical solution using the "upwind" scheme coded in Python**

Diffusion can be expressed in terms of advective fluxes:

$$ \partial_t {\color{green}\psi} +\partial_x \big(\, \underbrace{-\mu \frac{\partial_x {\color{green}\psi}}{{\color{green}\psi}}}_{u} \psi \big) = 0$$

hence we can leverage the basic [**upwind**](https://en.wikipedia.org/wiki/Upwind_scheme) solver to numerically integrate the diffusion equation: 

* ${\color{red}n} = \frac{\color{red}t}{\Delta t} \in 0, 1, \ldots$   
* ${\color{blue}i} = \frac{\color{blue}x}{\Delta x} \in 0, 1, \ldots$   
* ${\color{brown}C} = {\color{brown}u}\frac{\Delta t}{\Delta x}$ (so called Courant number)   
* ${\color{green}\psi}_{\color{blue}i}^{\color{red}n}$: scalar field at "grid cell" $\color{blue}i$ in space at time "level" ${\color{red}n}$   


$$
  {\color{green}\psi}^{{\color{red}n}+1}_{\color{blue}i} = {\color{green}\psi}^{\color{red}n}_{\color{blue}i} - \left[F\left({\color{green}\psi}^{\color{red}n}_{\color{blue}i},  {\color{green}\psi}^{\color{red}n}_{{\color{blue}i}+1}, {\color{brown}C_{i+1/2}^{\color{red}n}}\right)-F\left({\color{green}\psi}^{\color{red}n}_{{\color{blue}i}-1},{\color{green}\psi}^{\color{red}n}_{\color{blue}i}, {\color{brown}C_{i-1/2}^{\color{red}n}}\right)\right]
$$
$$F(\psi_L, \psi_R, C)=\frac{C+|C|}{2}\psi_L + \frac{C-|C|}{2}\psi_R$$

with:

$$C_{{\color{blue}i}-1/2}=\frac{\Delta t}{\Delta x}u_{{\color{blue}i}-1/2}=-\mu\frac{\Delta t}{\Delta x}\frac{{\color{green}\psi}_{\color{blue}i}-{\color{green}\psi}_{{\color{blue}i}-1}}{\Delta x}\frac{2}{{\color{green}\psi}_{{\color{blue}i}}+{\color{green}\psi}_{{\color{blue}i}-1}}$$

**4. upwind algorithm coded in Python/NumPy: functions and a useful structure**

In [ ]:
def F(psi_l, psi_r, C):
    return .5 * (C + abs(C)) * psi_l + \
           .5 * (C - abs(C)) * psi_r

class shift:
    pass # TODO

**5. upwind algorithm coded in Python/NumPy: data/objects**

In [ ]:
one = shift(add=1, sub=1)
hlf = shift(add=0, sub=1)

nt = 500
dt = t_max / nt

i = slice(1, nx-2)
im = slice(1, nx-1)

**6. upwind algorithm coded in Python/NumPy: time-stepping loop**

In [ ]:
psi = psi_analytic(x=x, t=0)
C = np.zeros(len(psi) - 1)
output = [psi.copy()]
for _ in range(nt + 1):
    # TODO
    assert max(abs(C)) <= 1 
    # TODO
    output += [psi.copy()]

**7. comparing analytic and numerical solutions**

In [ ]:
def plot(step):
    fig = pyplot.figure()
    pyplot.step(x, psi_analytic(x=x, t=0), label='initial (discretised)', where='mid')
    pyplot.step(x, psi_analytic(x=x, t=step*dt), label='analytical (discretised)', where='mid')
    pyplot.step(x, output[step], label=f'numerical (upwind) t/t_max={step}', where='mid')
    pyplot.legend()
    pyplot.grid()
    return fig 
plot(nt)
show_plot('numeric.pdf')

**8. animation**

In [ ]:
show_anim(plot, range(0, nt, 3))